In [2]:
import pandas as pd
from pathlib import Path
import pyreadstat
from tqdm import tqdm

In [5]:
Path.cwd()

PosixPath('/Users/rishaan/Desktop/CERAS-Cognitive-Efficiency-Reasoning-Alignment-System/data')

In [6]:
 # Base directory where extracted .sav files are stored
raw_pisa_dir = Path.cwd().parent / "data" / "raw"

# Output directory for parquet files
parquet_dir = Path.cwd().parent / "data" / "processed"
parquet_dir.mkdir(parents=True, exist_ok=True)

raw_pisa_dir, parquet_dir

(PosixPath('/Users/rishaan/Desktop/CERAS-Cognitive-Efficiency-Reasoning-Alignment-System/data/raw'),
 PosixPath('/Users/rishaan/Desktop/CERAS-Cognitive-Efficiency-Reasoning-Alignment-System/data/processed'))

In [4]:
list(raw_pisa_dir.glob("*.SAV"))

[PosixPath('/Users/rishaan/Desktop/CERAS-Cognitive-Efficiency-Reasoning-Alignment-System/data/raw/CY08MSP_STU_COG.SAV'),
 PosixPath('/Users/rishaan/Desktop/CERAS-Cognitive-Efficiency-Reasoning-Alignment-System/data/raw/CY08MSP_STU_QQQ.SAV')]

In [5]:
cog_path = raw_pisa_dir / "CY08MSP_STU_COG.SAV"
qqq_path = raw_pisa_dir / "CY08MSP_STU_QQQ.SAV"

In [6]:
def read_sav_in_chunks(path, chunksize=100_000):
    chunks = []
    
    reader = pyreadstat.read_file_in_chunks(
        pyreadstat.read_sav,
        path,
        chunksize=chunksize
    )
    
    for chunk, meta in tqdm(reader, desc=f"Reading {path.name}"):
        chunks.append(chunk)
    
    df = pd.concat(chunks, ignore_index=True)
    return df

In [7]:
df_cog = read_sav_in_chunks(cog_path, chunksize=100_000)
df_qqq = read_sav_in_chunks(qqq_path, chunksize=100_000)

df_cog.shape, df_qqq.shape

Reading CY08MSP_STU_COG.SAV: 7it [03:06, 26.63s/it]
Reading CY08MSP_STU_QQQ.SAV: 7it [01:27, 12.54s/it]


((613744, 5023), (613744, 1278))

In [8]:
df_cog.to_parquet(parquet_dir / "pisa_2022_student_cog.parquet")
df_qqq.to_parquet(parquet_dir / "pisa_2022_student_qqq.parquet")